In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
#from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [2]:
def format_df(filename):
    df = pd.read_csv(filename)
    df['harvest'] = pd.to_datetime(df['harvest'], format='%Y/%m/%d')
    df['planting'] = pd.to_datetime(df['planting'], format='%Y/%m/%d')
    df['date'] = df['harvest'].dt.strftime("%Y")
    df = df.rename(columns={df.columns[1]: "Region"})
    #df['Region'] = df['Region'].str.upper()
    return df

In [3]:
data = format_df("rheas/tanzania_tamsat_25km_unimode.csv")

In [4]:
print(data.head(3))

   gid   Region  ensemble    harvest   planting  wsgd  date
0   18  Buhigwe        10 2022-01-21 2021-09-22   0.0  2022
1   18  Buhigwe        27 2022-01-21 2021-09-22   0.0  2022
2   18  Buhigwe        19 2022-01-21 2021-09-22   0.0  2022


In [5]:
# Unimodal season:  October--August (LR)
lrStart = [9, 10, 11, 12, 1,2, 3, 4]
lrEnd = [2, 3, 4, 5, 6, 7, 8]
def LR_metrics(lr, lrStart, lrEnd, season):
    lr.loc[(pd.to_datetime(lr['planting']).dt.month.isin(lrStart)) & (pd.to_datetime(lr['harvest']).dt.month.isin(lrEnd)), 'Season'] = season
    lr = lr.groupby(['Region', 'date', 'Season'], as_index=False).agg({'wsgd': 'mean'})

    return lr

rh_lr = LR_metrics(data, lrStart, lrEnd, "LR")
print(rh_lr.head())

     Region  date Season      wsgd
0      Bahi  2022     LR  0.014275
1      Bahi  2023     LR  0.129525
2   Buhigwe  2022     LR  0.113337
3   Buhigwe  2023     LR  0.012735
4  Chamwino  2022     LR  0.089038


In [6]:
# Unimodal season:  October--August (OND)
ondStart = [9, 10, 11]
ondEnd = [1, 2, 3]
def OND_metrics(ond, ondStart, ondEnd, season):
    ond.loc[(pd.to_datetime(ond['planting']).dt.month.isin(ondStart)) & (pd.to_datetime(ond['harvest']).dt.month.isin(ondEnd)), 'Season'] = season
    ond = ond.groupby(['Region', 'date', 'Season'], as_index=False).agg({'wsgd': 'mean'})

    return ond

rh_ond = OND_metrics(data, ondStart, ondEnd, "OND")
print(rh_ond.head())

    Region  date Season      wsgd
0     Bahi  2022     LR  0.028550
1     Bahi  2022    OND  0.000000
2     Bahi  2023     LR  0.259050
3     Bahi  2023    OND  0.000000
4  Buhigwe  2022     LR  0.277675


In [ ]:
geo_source = GeoJSONDataSource(geojson=merged_data.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [ ]:
color_mapper = LinearColorMapper(palette=Viridis256, low=min(merged_data[gwad]), high=max(merged_data[gwad]))

In [ ]:
p1 = figure(title=Yield Data by District,
           plot_width=650, plot_height=600)

p1.multi_line(xs,ys, source=country_geo_source,
            line_color=black, line_width=0.5)

p1.patches(xs,ys, source=geo_source,
          fill_color={field: gwad, transform: color_mapper},
          fill_alpha=0.7, line_color=black, line_width=0.5)

hover = HoverTool(tooltips=[(District, @NAME_2), (gwad, @gwad)])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation=vertical)
p1.add_layout(color_bar, right)

In [ ]:
color_mapper1 = LinearColorMapper(palette=Viridis256, low=min(merged_data[wsgd]), high=max(merged_data[wsgd]))

p2 = figure(title=Yield Data by District,
           plot_width=650, plot_height=600)

p2.multi_line(xs,ys, source=country_geo_source,
            line_color=black, line_width=0.5)

p2.patches(xs,ys, source=geo_source,
          fill_color={field: wsgd, transform: color_mapper1},
          fill_alpha=0.7, line_color=black, line_width=0.5)

hover = HoverTool(tooltips=[(District, @NAME_2), (wsgd, @wsgd)])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation=vertical)
p2.add_layout(color_bar, right)

In [ ]:
from bokeh.layouts import column
layout = column(p1, p2)
#show(layout)

In [ ]:
from bokeh.io import output_file, show
output_file(plot.html)
show(layout)

In [ ]:
#from bokeh.io import save

# specify the output file name and location
#save(p, filename=yield_map.html, title=Yield Data Map)